*machine learning algorithm for conversation*

In [1]:
import tensorflow as tf
import pickle
import json
import re


# imoport the spaCy nlp model
import spacy
nlp = spacy.load('en_core_web_sm')

The json containing the training data is loaded here - this file is not mine but I will eventually write my own data as this file doesn't fit my needs.

In [2]:
with open('intents.json') as f:
    intents = json.load(f)

Raw data is to be cleansed and tokenised

In [3]:
def clean_str(txt):
    txt = txt.lower()
    txt = re.sub(r'[^a-zA-z.?!\']', ' ', txt)
    txt = re.sub(r'[ ]+', ' ', txt)
    
    specials = ['.', '!', '?']
    for s in specials:
        txt = txt.replace(s, f' {s} ')
    
    return txt

def tokenise_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)
    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq)
    
    return tokenizer, input_seq

Information is extracted from the json file

In [4]:
inputs, targets = [], []   # all inputs, corresponding intent by position in list
classes = []               # complete set of intents
intent_doc = {}            # intent: list[outputs]

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(clean_str(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)
        
# tokenise input data
tokenizer, input_tensor = tokenise_data(inputs)

Construct a tokeniser dictionary

In [5]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

Define constants to be used by the ml model

In [6]:
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

Build RNN model

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         61440     
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 753,558
Trainable params: 753,558
Non-trainable params: 0
__________________________________________________

Train the model

In [8]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 3s 36ms/step - loss: 3.0831 - accuracy: 0.0979
Epoch 2/50
5/5 [==============================] - 0s 32ms/step - loss: 2.5790 - accuracy: 0.3007
Epoch 3/50
5/5 [==============================] - 0s 39ms/step - loss: 1.8148 - accuracy: 0.4545
Epoch 4/50
5/5 [==============================] - 0s 31ms/step - loss: 1.2234 - accuracy: 0.6014
Epoch 5/50
5/5 [==============================] - 0s 38ms/step - loss: 0.8295 - accuracy: 0.7203
Epoch 6/50
5/5 [==============================] - 0s 36ms/step - loss: 0.4271 - accuracy: 0.8601
Epoch 7/50
5/5 [==============================] - 0s 32ms/step - loss: 0.3671 - accuracy: 0.8671
Epoch 8/50
5/5 [==============================] - 0s 42ms/step - loss: 0.1626 - accuracy: 0.9510
Epoch 9/50
5/5 [==============================] - 0s 32ms/step - loss: 0.1657 - accuracy: 0.9301
Epoch 10/50
5/5 [==============================] - 0s 33ms/step - loss: 0.0958 - accuracy: 0.9720
Epoch 11/50
5/5 [============

Finally, save the model and relevant data

In [9]:
model.save('chatbot_model.h5', save_format='h5')

to_save = [tokenizer, intent_doc, trg_index_word]
file_name = ['tokenizer', 'intent_doc', 'trg_index_word']

for i in range(len(to_save)):
    with open(f'pickles/{file_name[i]}.pkl', 'wb') as file:
        pickle.dump(to_save[i], file)